**Created on**: 2021.09.04

**Implemented by**:Anthony Cho

**Subject**: Linear programming: Insurance company

**Reference**: Introduction to operation research (10th Edition) - Hillier and Lieberman.
**Problem**: 3.1-9

## Problem: 

The Primo Insurance Company is introducing two new product lines: special risk insurance and mortgages. The expected profit is: 
* &#36;5 per unit on special risk insurance 
* &#36;2 per unit on mortgages. 

Management wishes to establish sales quotas for the new product lines to maximize total expected profit. The work requirements are as follows:

|                  | Work-hours per unid   |              |                           |
|------------------|-----------------------|--------------|---------------------------|
| **Department**   | **Special Risk**      | **Mortgage** | **Work-Hours Available**  |
| Underwriting     | 3                     | 2            | 2400                      |
| Administration   | 0                     | 1            | 800                       |
| Claims           | 2                     | 0            | 1200                      |

**Parameters**:
* $g_j$: expected profit per unit by product $j$.
* $w_{i,j}$: amount of work-hours per unit by product $j$ processed in departament $i$.
* $l_i$: work-hours available in department $i$.

**Decision variables**:
* $X_j$: amount of units to sell the product $j$.

**Constraints**:
* **work-hours limit**: $\quad \sum_{j} w_{i,j} \cdot X_{j} \leq l_j, \quad \forall i.$
* **Decision variable bound**: $X_{j} \geq 0, \quad \forall j$.

**Objective function**:
* $\max_{X} \sum_{j} g_{j} \cdot X_{j}$.


### General model

$$\max_{X} \sum_{j} g_{j} \cdot X_{j}$$

s.t.

$$\quad \sum_{j} w_{i,j} \cdot X_{j} \leq l_j, \quad \forall i.$$
$$X_{j} \geq 0, \quad \forall j$$

### Particular model

$$\max_{X} 5 X_{1} + 2 X_{2}$$

s.a.

|||||
|----------|---------  |--------|------|
| $3X_{1}$  | $+2X_{2}$ | $\leq$ | 2400 |
|          | $X_{2}$   | $\leq$ | 800  |
| $2X_{1}$ |           | $\leq$ | 1200 |


$$X_{1}, X_{2} \geq 0$$

In [1]:
## Libraries dependencies

import gurobipy as gp

### Parameters

In [2]:
## Expected profit per unit by product
profit = {'Special Risk': 5, 
            'Mortgage': 2}

## Work-hours per product by department
W = {'Underwriting': {'Special Risk': 3, 'Mortgage': 2},
     'Administration': {'Special Risk': 0, 'Mortgage': 1},
     'Claims': {'Special Risk': 2, 'Mortgage': 0}
    }

## Work-hours available per department
L = {'Underwriting': 2400,
     'Administration': 800,
     'Claims': 1200
    }

### LP-Model

In [3]:
## Model instance
model = gp.Model('InsuranceCompany')
model.modelSense = gp.GRB.MAXIMIZE

Academic license - for non-commercial use only - expires 2022-11-29
Using license file /home/hp/gurobi.lic


In [4]:
## Decision-variables
X = {}
for p in profit.keys():
    X[p] = model.addVar(obj=profit[p], lb=0, vtype=gp.GRB.CONTINUOUS, name=f'X[{p}]')

In [5]:
## Constraints
for dep in W.keys():
    lexp = gp.LinExpr()
    for p in W[dep].keys():
        lexp.addTerms(W[dep][p], X[p])
    model.addConstr(lexp, gp.GRB.LESS_EQUAL, L[dep], name=f'MaxWorkHours[{dep}]')

In [6]:
## Model update
model.update()

In [7]:
## Model display
model.display()

Maximize
   <gurobi.LinExpr: 5.0 X[Special Risk] + 2.0 X[Mortgage]>
Subject To
   MaxWorkHours[Underwriting] : <gurobi.LinExpr: 3.0 X[Special Risk] + 2.0 X[Mortgage]> <= 2400.0
   MaxWorkHours[Administration] : <gurobi.LinExpr: X[Mortgage]> <= 800.0
   MaxWorkHours[Claims] : <gurobi.LinExpr: 2.0 X[Special Risk]> <= 1200.0


In [8]:
## Optimize model
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x90eb4f53
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+02, 2e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+03   5.000000e+01   0.000000e+00      0s
       1    3.6000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  3.600000000e+03


#### Post-Processing: Interpretation

In [9]:
if model.status == gp.GRB.OPTIMAL:
    for p in X.keys():
        if X[p].x:
            print('Sell {} units of product {}'.format(X[p].x, p))

    print(f'\nTotal profit ${model.ObjVal}\n')
    
    for c in model.getConstrs():
        print('Slack {:30}: {:>8.2f}'.format(c.ConstrName, c.slack))
    
else:
    print('No solution had found.')

Sell 600.0 units of product Special Risk
Sell 300.0 units of product Mortgage

Total profit $3600.0

Slack MaxWorkHours[Underwriting]    :     0.00
Slack MaxWorkHours[Administration]  :   500.00
Slack MaxWorkHours[Claims]          :     0.00
